In [1]:
import time
import csv
from IPython.display import display, Markdown, Latex
import networkx as nx
import bravo.regulation as bravo_reg
import pyBravo as bravo_main

def read_input_genes(filename):
    res = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            res.append(''.join(row))
    return res

--- Memory foot print cache ---
index_syn size : 244398
index_std size : 161611


## Upstream regulation network reconstruction

This call would test the regulation network reconstruction on all available data sources, and stop at depth 2.  
`reconstructed_network = bravo.upstream_regulation(['JUN/FOS', 'SCN5A'], max_depth=2)`

This call would test the regulation network reconstruction on `reactome` and `msigdb` data sources, stopping when no new regulations can be found.   
`reconstructed_network = bravo.upstream_regulation(['JUN/FOS', 'SCN5A'], data_sources=['reactome', 'msigdb'])`

In [2]:
start_time = time.time()

# global options
bravo_reg.DECOMPOSE_COMPLEXES = True
bravo_reg.EXTEND_WITH_SYNONYMS = True
bravo_reg.EXTEND_WITH_SUFFIXES = True
bravo_reg.VERVOSE = False

""" all possible data sources """
ds = ['bind', 'biogrid', 'corum',
                'ctd', 'dip', 'drugbank', 'hprd', 'humancyc', 'inoh',
                'intact', 'kegg', 'mirtarbase', 'netpath', 'panther',
                'pid', 'psp', 'reactome', 'reconx', 'smpdb', 'wp',
                'intact_complex', 'msigdb']

""" removing mirtarbase """
target_data_sources = set(ds) - set(['mirtarbase'])

""" specifiyng inputs """
gene_list = ['JUN/FOS', 'SCN5A']
#gene_list = read_input_genes('iLiverCancer1715.target')

""" reconstruction call """
reconstructed_network = bravo_reg.upstream_regulation(gene_list, 
                                                      data_sources=target_data_sources, 
                                                      max_depth=2)

elapsed_time = round((time.time() - start_time), 2)

print("--- Upstream regulation network in %s seconds ---" % elapsed_time)


exploration depth 0
to be explored ['JUN/FOS', 'SCN5A']
JUN/FOS decomposed into ['JUN', 'FOS']
to be explored after complex decomposition ['JUN/FOS', 'SCN5A', 'JUN', 'FOS']
new synonmys to be explored:['CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'AP-1', 'AP1', 'c-Jun', 'p39', 'AP-1', 'C-FOS', 'p55']
exploring ['JUN/FOS', 'SCN5A', 'JUN', 'FOS', 'CDCD2', 'CMD1E', 'CMPD2', 'HB1', 'HB2', 'HBBD', 'HH1', 'ICCD', 'IVF', 'LQT3', 'Nav1.5', 'PFHB1', 'SSS1', 'VF1', 'AP-1', 'AP1', 'c-Jun', 'p39', 'C-FOS', 'p55', 'JUN/FOS mRna', 'JUN/FOS protein', 'SCN5A mRna', 'SCN5A protein', 'JUN mRna', 'JUN protein', 'FOS mRna', 'FOS protein', 'CDCD2 mRna', 'CDCD2 protein', 'CMD1E mRna', 'CMD1E protein', 'CMPD2 mRna', 'CMPD2 protein', 'HB1 mRna', 'HB1 protein', 'HB2 mRna', 'HB2 protein', 'HBBD mRna', 'HBBD protein', 'HH1 mRna', 'HH1 protein', 'ICCD mRna', 'ICCD protein', 'IVF mRna', 'IVF protein']
		removing suffix  complex for NSC 689534/Cop


Explored 65 regulators
exploring ['ICER', 'NGF', 'BDNF', 'IL2', 'GHRL', 'APP', 'CSF1', 'GAST', 'Histamine/HRH2', 'Win 55212-2/CNR1', '3-(2-hydroxy-4-(1,1-dimethylheptyl)phenyl)-4-(3-hydroxypropyl)cyclohexanol/CNR1', 'ATF2/c-Jun', 'SRF(dimer)', 'MAL(dimer)', 'JUN', 'ERK1-2', 'NSC 689534', 'Copper', 'Clioquinol', 'Dimaprit', 'HRH2', 'Organoselenium Compounds analog', 'Disulfiram', 'TCR', 'CD3', 'MHC II', 'CD4', 'IL12', 'IL12R', 'TYK2', 'JAK2', 'RHOA', 'GTP', 'SMAD3', 'SMAD4', 'FOS', 'Histamine', 'Win 55212-2', 'CNR1', '3-(2-hydroxy-4-(1,1-dimethylheptyl)phenyl)-4-(3-hydroxypropyl)cyclohexanol', 'c-Jun', 'FOXM1', 'FKHL16', 'FOXM1A', 'FOXM1B', 'HFH-11', 'HFH11', 'HNF-3', 'INS-1', 'MPHOSPH2']
		removing suffix  complex for NSC 689534/Copper complex
				 --> NSC 689534/Copper
		removing suffix  protein for SLC6A3 protein mutant form
				 --> SLC6A3 mutant form
		removing suffix  mutant form for SLC6A3 mutant form
				 --> SLC6A3
		removing suffix  protein for SOD2 protein
				 --> SOD2
		re

		removing suffix  protein for IFNE protein
				 --> IFNE
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  protein for IL1A protein
				 --> IL1A
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  protein for PPARA protein
				 --> PPARA
		removing suffix  protein for F2 protein
				 --> F2
		removing suffix  protein for AGT protein
				 --> AGT
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  protein for NOG protein
				 --> NOG
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  protein for NOG protein
				 --> NOG
		removing suffix  protein for GDNF protein
				 --> GDNF
		removing suffix  protein for NOG protein
				 --> NOG
		removing suffix  protein for F2 protein
				 --> F2
		removing suffix  protein for INS protein
				 --> INS
		removing suffix  protein for FGF2 protein
				 --> FGF2
		removing suffix  mutant form


Explored 202 regulators
exploring ['AGXT1 mRna', 'AGXT1 protein', 'PH1 mRna', 'PH1 protein', 'SPAT mRna', 'SPAT protein', 'SPT mRna', 'SPT protein', 'TLH6 mRna', 'TLH6 protein', 'HOMG4 mRna', 'HOMG4 protein', 'URG mRna', 'URG protein', 'DADR mRna', 'DADR protein', 'DRD1A mRna', 'DRD1A protein', 'CDF mRna', 'CDF protein', 'DIA mRna', 'DIA protein', 'HILDA mRna', 'HILDA protein', 'MLPLI mRna', 'MLPLI protein', 'D3DR mRna', 'D3DR protein', 'ETM1 mRna', 'ETM1 protein', 'FET1 mRna', 'FET1 protein', 'IBGC5 mRna', 'IBGC5 protein', 'PDGF-2 mRna', 'PDGF-2 protein', 'PDGF2 mRna', 'PDGF2 protein', 'SIS mRna', 'SIS protein', 'SSV mRna', 'SSV protein', 'c-sis mRna', 'c-sis protein', 'TFGA mRna', 'TFGA protein', 'IL-1A mRna', 'IL-1A protein', 'IL1 mRna', 'IL1 protein']

Explored 202 regulators
exploring ['IL1-ALPHA mRna', 'IL1-ALPHA protein', 'IL1F1 mRna', 'IL1F1 protein', 'ATF mRna', 'ATF protein', 'HFB1-GDNF mRna', 'HFB1-GDNF protein', 'HSCR3 mRna', 'HSCR3 protein', 'CT-1 mRna', 'CT-1 protein', '

## Metrics and SIF export of the regulation network

In [3]:
G = bravo_main.build_nx_digraph(reconstructed_network)

Number of nodes = 248
Number of edges = 323


In [4]:
G_unified = bravo_reg.fast_reg_network_unification(G, bravo_reg.index_syn)
print('Nodes after synonym-based unification = ' + str(len(G_unified.nodes())))
print('Edges after synonym-based unification = ' + str(len(G_unified.edges())))

		merging node IL-3 into node IL3
		merging node DAT into node SLC6A3
		merging node c-Jun into node JUN
		merging node c-Jun into node JUN
		merging node ATF1 into node GDNF
		merging node ATF2 into node GDNF
		merging node ATF1 into node GDNF
		merging node AP1 into node JUN
		merging node AP1 into node JUN
		merging node ATF2 into node GDNF
		merging node MNSOD into node SOD2
		merging node FOXO3A into node FOXO3
		merging node p300 into node EP300
		merging node RANKL into node TNFSF11
		merging node TNF-alpha into node TNF
		merging node p300 into node EP300
Nodes after synonym-based unification = 237
Edges after synonym-based unification = 321


In [5]:
md = bravo_main.get_centrality_as_md(G_unified)
display(Markdown(md))


| Gene | Degree Centrality |
|------|------|
| TNF | 0.369 | 
| FOS | 0.242 | 
| SOD2 | 0.191 | 
| IL2 | 0.072 | 
| IL1A | 0.068 | 
| JUN | 0.059 | 
| FGF2 | 0.055 | 
| EP300 | 0.047 | 
| NGF | 0.047 | 
| BDNF | 0.047 | 


In [6]:
bravo_main.write_to_SIF(G_unified, 'tutorial.sif')
bravo_main.write_provenance(G_unified, 'tutorial-prov.csv')

SIF network written to tutorial.sif
Basic regulation reaction provenance written to tutorial-prov.csv
